# Class for Grid Search

In [113]:
# external libraries
import numpy as np
# local libraries
from estimator import Estimator
from util_classes import Dataset
from nn import NeuralNetwork, LinearLayer, ActivationFunction

from loss import LossFunction

class GridSearch():
    def __init__(self, estimator: Estimator, hyper_grid: dict):
        self.estimator = estimator
        self.hyper_grid = hyper_grid

    #returns a list of data folds through indexes
    def _generate_folds(self) -> list(tuple(Dataset, Dataset)):
        n_folds = self._n_folds
        dataset = self._dataset
        print(dataset.ids)
        data_size = dataset.ids.shape[0]
        indices = np.arange(data_size)
        
        #TODO maybe shuffle not needed if we assume dataset has already been shuffled
        np.random.shuffle(indices)
        
        folds = []

        for index_lists in np.array_split(indices, n_folds):
            mask = np.zeros(data_size, dtype=bool)
            mask[index_lists] = True
            test_indices = indices[mask]
            train_indices = indices[~mask]
            test_set = Dataset(ids=dataset.ids[test_indices], labels=dataset.labels[test_indices], data=dataset.data[test_indices])
            train_set = Dataset(ids=dataset.ids[train_indices], labels=dataset.labels[train_indices], data=dataset.data[train_indices])
            folds.append((train_set, test_set))
        return(folds)

    #returns the best set of hyperparameters
    def k_fold(self, dataset: Dataset, n_folds: int):
        if(isinstance(dataset) == Dataset):
            self._dataset = dataset
        else:
            raise TypeError
        
        if(type(n_folds) == int):
            self._n_folds = n_folds
        else:
            raise TypeError
        
        data_size = dataset.ids.shape[0]
        if(n_folds > data_size):
            raise ValueError
            
        self._generate_folds()
        #for loop iterating all combinations
            #give combination to estimator to initialize new NN
            #train on TR set
            #evaluate on VL set
            #save results for combination
        #return best combination
            
            
    #returns an estimation of the risk for the model, average +- variance
    def nested_k_fold(dataset: Dataset, inner_n_folds:int, outer_n_folds:int):
        print("hello")

In [114]:
net = NeuralNetwork([
    LinearLayer((8, 16)),
    ActivationFunction(),
    LinearLayer((16, 16)),
    ActivationFunction(),
    LinearLayer((16, 2))
])
estimator = Estimator(net)
grid = {}
grid['eta'] = [0.1, 0.2, 0.3]
grid['momentum'] = [0.1, 0.2, 0.3]


In [115]:
from datasets import read_monks
data = read_monks(1, "train")
ids = data.ids.copy()
data.shape

(124, [6, 1])

In [116]:
selector = GridSearch(estimator, grid)
selector.k_fold(data, 5)

[  5   6  19  22  27  28  37  39  42  50  51  53  56  57  61  62  64  67
  72  76  86  87  88  92  93  94  99 103 107 111 114 116 117 119 120 124
 130 132 134 135 136 137 139 143 144 149 150 153 154 156 157 159 160 167
 172 173 176 181 184 188 191 195 196 197 206 209 210 212 214 216 217 222
 223 224 227 239 241 249 253 258 261 264 270 273 274 275 286 289 290 297
 300 308 313 316 324 326 332 337 344 346 352 361 362 366 377 379 383 385
 387 392 398 400 402 403 404 408 409 414 415 416 426 428 430 432]
[(Dataset(ids=array([264, 130, 214, 430, 324,   6, 114,  50,  64, 392, 172, 366, 344,
       409, 124,  72,  19, 197, 153, 149, 352, 117, 332, 273,  57,  37,
       379, 313, 253, 286, 274, 154, 134, 239, 224, 119, 377, 188, 206,
       404, 361,  88, 156,  62,  53, 181, 398, 261,  51,   5, 157, 116,
       249, 290, 227, 135, 212, 191, 137,  56, 120, 337, 416,  94, 150,
       403, 362,  39, 241, 428,  22, 289, 400,  27, 346,  67,  86, 167,
       383, 111,  61, 402, 308, 415,  42,  87, 159

In [24]:
test = np.array([0,1,2,3,4,5,6,7,8,9])
print(test)
perm = np.random.permutation(10)


[0 1 2 3 4 5 6 7 8 9]
[7 5 1 4 3 6 2 0 9 8]


In [42]:
size = 13
n_folds = 5
r = size % n_folds
q = size // n_folds
for i in range(n_folds):
    print(np.arange)
print(np.arange(0, q + 1))


[0 1 2]


In [51]:
tmp = np.arange(10)
test = tmp.copy()
np.random.shuffle(tmp)
print(tmp)
for i in np.array_split(tmp, 3):
    print(test[i])

[1 8 5 9 7 0 2 6 3 4]
[1 8 5 9]
[7 0 2]
[6 3 4]
